# Main notebook for battery state estimation

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback


IS_COLAB = False

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/battery-state-estimation/battery-state-estimation/"
else:
    data_path = "../"

sys.path.append(data_path)
from data_processing.unibo_powertools_data import UniboPowertoolsData, CycleCols
from data_processing.model_data_handler import ModelDataHandler

Using TensorFlow backend.


### Config logging

In [2]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

### Initial the data object

Load the cycle and capacity data to memory based on the specified chunk size

In [3]:
dataset = UniboPowertoolsData(
    test_types=['S'],
    chunk_size=1000000,
    lines=[37, 40],
    charge_line=37,
    discharge_line=40,
    base_path=data_path
)

2020/12/02 17:47:14 [DEBUG]: Start loading data with lines: [37, 40], types: ['S'] and chunksize: 1000000...
2020/12/02 17:47:40 [DEBUG]: Finish loading data.
2020/12/02 17:47:40 [INFO]: Loaded raw dataset A data with cycle row count: 6181604 and capacity row count: 16548
2020/12/02 17:47:40 [DEBUG]: Start cleaning cycle raw data...
2020/12/02 17:47:45 [DEBUG]: Finish cleaning cycle raw data.
2020/12/02 17:47:45 [INFO]: Removed 5 rows of abnormal cycle raw data.
2020/12/02 17:47:45 [DEBUG]: Start cleaning capacity raw data...
2020/12/02 17:47:45 [DEBUG]: Finish cleaning capacity raw data.
2020/12/02 17:47:45 [INFO]: Removed 1 rows of abnormal capacity raw data.
2020/12/02 17:47:45 [DEBUG]: Start assigning charging raw data...
2020/12/02 17:47:46 [DEBUG]: Finish assigning charging raw data.
2020/12/02 17:47:46 [INFO]: [Charging] cycle raw count: 4773746, capacity raw count: 8278
2020/12/02 17:47:46 [DEBUG]: Start assigning discharging raw data...
2020/12/02 17:47:46 [DEBUG]: Finish assi

### Determine the training and testing name

Prepare the training and testing data for model data handler to load the model input and output data.

In [4]:
train_data_test_names = [
    '000-DM-3.0-4019-S', 
    '001-DM-3.0-4019-S', 
    '002-DM-3.0-4019-S', 
    '006-EE-2.85-0820-S', 
    '007-EE-2.85-0820-S', 
    '018-DP-2.00-1320-S', 
    '019-DP-2.00-1320-S',
    '036-DP-2.00-1720-S', 
    '037-DP-2.00-1720-S', 
    '038-DP-2.00-2420-S', 
    '040-DM-4.00-2320-S',
    '042-EE-2.85-0820-S', 
    '045-BE-2.75-2019-S'
]

test_data_test_names = [
    '003-DM-3.0-4019-S',
    '008-EE-2.85-0820-S',
    '039-DP-2.00-2420-S', 
    '041-DM-4.00-2320-S',    
]

dataset.prepare_data(train_data_test_names, test_data_test_names)

2020/12/02 17:47:46 [DEBUG]: Start preparing data for training: ['000-DM-3.0-4019-S', '001-DM-3.0-4019-S', '002-DM-3.0-4019-S', '006-EE-2.85-0820-S', '007-EE-2.85-0820-S', '018-DP-2.00-1320-S', '019-DP-2.00-1320-S', '036-DP-2.00-1720-S', '037-DP-2.00-1720-S', '038-DP-2.00-2420-S', '040-DM-4.00-2320-S', '042-EE-2.85-0820-S', '045-BE-2.75-2019-S'] and testing: ['003-DM-3.0-4019-S', '008-EE-2.85-0820-S', '039-DP-2.00-2420-S', '041-DM-4.00-2320-S']...
2020/12/02 17:47:57 [DEBUG]: Finish getting training and testing charge data.
2020/12/02 17:48:06 [DEBUG]: Finish getting training and testing discharge data.
2020/12/02 17:48:06 [DEBUG]: Finish cleaning training and testing charge data.
2020/12/02 17:48:06 [DEBUG]: Finish cleaning training and testing discharge data.
2020/12/02 17:48:07 [DEBUG]: Finish adding training and testing discharge SOC parameters.
2020/12/02 17:48:12 [DEBUG]: Finish adding training and testing discharge SOH parameters.
2020/12/02 17:48:12 [DEBUG]: Finish preparing da

### Initial the model data handler

Model data handler will be used to get the model input and output data for further training purpose.

In [5]:
mdh = ModelDataHandler(dataset, [
    CycleCols.VOLTAGE,
    CycleCols.CURRENT,
    CycleCols.TEMPERATURE
])

# Data loading

In [6]:
train_x, train_y, test_x, test_y = mdh.get_discharge_whole_cycle(soh = False, output_capacity = True)

2020/12/02 17:48:18 [INFO]: Train x: (6536, 287, 3), train y: (6536, 287, 2) | Test x: (1728, 287, 3), test y: (1728, 287, 2)


In [7]:
train_y = mdh.keep_only_capacity(train_y, is_multiple_output = True)
test_y = mdh.keep_only_capacity(test_y, is_multiple_output = True)

2020/12/02 17:48:18 [INFO]: New y: (6536, 287)
2020/12/02 17:48:18 [INFO]: New y: (1728, 287)


In [8]:
experiment_name = '2020-12-02-12-28-17_lstm_soc'

history = pd.read_csv(data_path + 'results/trained_model/%s_history.csv' % experiment_name)
model = keras.models.load_model(data_path + 'results/trained_model/%s.h5' % experiment_name)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 287, 256)          266240    
_________________________________________________________________
lstm_1 (LSTM)                (None, 287, 256)          525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 287, 128)          197120    
_________________________________________________________________
dense (Dense)                (None, 287, 64)           8256      
_________________________________________________________________
dense_1 (Dense)              (None, 287, 1)            65        
Total params: 996,993
Trainable params: 996,993
Non-trainable params: 0
_________________________________________________________________


# Prediction time

In [12]:
prediction_time = []
for x in test_x:
  start = time.time()
  model.predict(x.reshape(1, x.shape[0], x.shape[1]))
  end = time.time()
  prediction_time.append(end - start)

prediction_time = np.array(prediction_time)
print('(Prediction) Average time: {} s, std: {} s, max: {} s, min: {} s'.format(
    prediction_time.mean(), prediction_time.std(), prediction_time.max(), prediction_time.min()))

(Prediction) Average time: 0.07088805155621634 s, std: 0.009244483929890438 s, max: 0.27985405921936035 s, min: 0.06615209579467773 s
